In [1]:
import json
from sage.all import GF, EllipticCurve

# 1. Cargar los parámetros de curvas
with open('results.json', 'r') as f:
    results = json.load(f)

# Extraer el entry de 32 bits
res32 = results['32'] if '32' in results else results[32]
p   = int(res32['p'])
a   = int(res32['a'])
b   = int(res32['b'])
G_xy = tuple(res32['G_xy'])

# Reconstruir el campo, la curva y el generador
F = GF(p)
E = EllipticCurve(F, [a, b])
G = E(G_xy)

print(f"Curva 32-bits: y^2 = x^3 + ({a})x + ({b}) over GF({p})")
print(f"Generador G = {G}")

# 2. Cargar los resultados ECDH
with open('ecdh_results.json', 'r') as f:
    ecdh_data = json.load(f)

# Buscar la entrada de 32 bits
entry = next(e for e in ecdh_data if int(e['bit_size']) == 32)
alice_priv = int(entry['alice_priv'])
bob_priv   = int(entry['bob_priv'])

# 3. Calcular las claves públicas
alice_pub = alice_priv * G
bob_pub   = bob_priv   * G

# 4. Mostrar por pantalla
print("\n=== ECDH en curva de 32 bits ===")
print(f"Alice private key: {alice_priv}")
print(f"Bob   private key: {bob_priv}")
print(f"Alice public key:  {alice_pub}")
print(f"Bob   public key:  {bob_pub}")


Curva 32-bits: y^2 = x^3 + (51016516)x + (2185363664) over GF(2786035199)
Generador G = (672793040 : 2269103569 : 1)

=== ECDH en curva de 32 bits ===
Alice private key: 1191513438
Bob   private key: 988230802
Alice public key:  (1111960910 : 654118817 : 1)
Bob   public key:  (670139701 : 179721383 : 1)


In [3]:
# 3. Resolver ECDLP usando la función integrada
d_alice = alice_pub.log(G)
d_bob   = bob_pub.log(G)

print("\n=== Recuperación de privadas con discrete_log ===")
print(f"Encontrado alice_priv = {d_alice}  (original = {alice_priv})")
print(f"Encontrado bob_priv   = {d_bob}    (original = {bob_priv})")


=== Recuperación de privadas con discrete_log ===
Encontrado alice_priv = 1191513438  (original = 1191513438)
Encontrado bob_priv   = 988230802    (original = 988230802)


In [18]:
from sage.all import GF, EllipticCurve
import hashlib, base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad

def derive_aes_key(shared_secret: int) -> bytes:
    """
    Derive a 256-bit AES key from the ECDH shared secret integer
    by hashing it with SHA-256.
    """
    # Ensure it's a Python int
    if isinstance(shared_secret, list):
        # If it's a list [x,y], take x (or combine as needed)
        shared_secret = shared_secret[0]
    # Convert secret to big-endian bytes (at least 1 byte)
    length = max(1, (shared_secret.bit_length() + 7) // 8)
    b = shared_secret.to_bytes(length, 'big')
    # Derive 32-byte key
    return hashlib.sha256(b).digest()

iv_b64 = "89PuzxknoQXGUEHByJ9wlg=="
cipher_b64 = "TUQzcBuCTpRmYzX3ZKOax3+eUnN6Knuv8xXeLZ9vDuoYwLtUgFK7Fp1jzUt80I8u/NMhnFcLTMEyS7F2ue/ODqLSwlfV2xC/rbG9hHOdO0c/sHi0RWI8nSS17VaTLPvmv74IVzC+zQR659R03363jg=="

shared_point = (alice_priv * bob_priv) * G
shared_ints = [int(shared_point.xy()[0]), int(shared_point.xy()[1])]

# ————— Derivación de clave AES-256 —————
# Convierte entero a bytes big-endian
aes_key = derive_aes_key(shared_int)
# Compare with 0111409ecbb111550f23de3b1eec810ede39d75f03710e29296dbb03396c7cbe
print(aes_key.hex())

# ————— Decodificar IV y ciphertext —————
iv = base64.b64decode(iv_b64)
ct = base64.b64decode(cipher_b64)

# ————— Descifrar con AES-CBC + PKCS#7 —————
cipher    = AES.new(aes_key, AES.MODE_CBC, iv)
plaintext = unpad(cipher.decrypt(ct), AES.block_size)

print("Mensaje descifrado:")
print(plaintext.decode('utf-8'))

dd3f403cdb81ef2b99f928bc4e3c327ab5f158aea2873562a67e6ec992831efc


ValueError: Padding is incorrect.